In [1]:
# Installing the required libraries
import pandas as pd
from datetime import datetime, timedelta

# Function to fill missing data in a column with the previous value
def fill_missing_data(row, column, prev_value):
    if pd.isnull(row[column]):
        return prev_value
    return row[column]

# Function to derive effective date and end date for a historical record
def derive_dates(start_date, end_date):
    if pd.isnull(end_date):
        return start_date, "2100-01-01"
    else:
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        return start_date.strftime("%Y-%m-%d"), (end_date - timedelta(days=1)).strftime("%Y-%m-%d")

# Reading input CSV file (input.csv)
input_file = "input.csv"
data = pd.read_csv(input_file)

# Creating an empty list to store transformed data
transformed_data = []

for index, row in data.iterrows():   # Processing each row of the input data
    # Extract employee information
    employee_code = row["Employee Code"]
    manager_employee_code = row["Manager Employee Code"]
    start_date = row["Date of Joining"]
    end_date = row["Date of Exit"]
    last_compensation = None
    last_pay_raise_date = None
    
    for i in range(1, 3):   # Iterating over each compensation, review, and engagement data
        # Filling missing data and deriving the effective dates
        compensation = fill_missing_data(row, f"Compensation {i}", last_compensation)
        compensation_date = fill_missing_data(row, f"Compensation {i} date", last_pay_raise_date)
        review = fill_missing_data(row, f"Review {i}", None)
        review_date = fill_missing_data(row, f"Review {i} date", None)
        engagement = fill_missing_data(row, f"Engagement {i}", None)
        engagement_date = fill_missing_data(row, f"Engagement {i} date", None)
        
        # Deriving effective date and end date
        effective_date, end_date = derive_dates(start_date, end_date)
        
        # Appending transformed data to the list
        transformed_data.append([
            employee_code, manager_employee_code, last_compensation, compensation, last_pay_raise_date,
            0,  # Assuming 'Variable Pay' is not provided in the input data
            0,  # Assuming 'Tenure in Org' is not provided in the input data
            review, review_date, engagement, engagement_date, effective_date, end_date
        ])
        
        # Updating the last compensation and last pay raise date for the next iteration
        last_compensation = compensation
        last_pay_raise_date = compensation_date

# Creating a DataFrame from the transformed data
transformed_df = pd.DataFrame(transformed_data, columns=[
    "Employee Code", "Manager Employee Code", "Last Compensation", "Compensation", "Last Pay Raise Date", "Variable Pay", "Tenure in Org", "Performance Rating", "Review Date", "Engagement Score", "Engagement Date", "Effective Date", "End Date"
])

# Writing the transformed data to a new CSV file (output.csv)
output_file = "output.csv"
transformed_df.to_csv(output_file, index=False)


In [2]:
df = pd.read_csv('output.csv')
print(df.head())

   Employee Code  Manager Employee Code  Last Compensation  Compensation  \
0              1                    NaN                NaN           NaN   
1              1                    NaN                NaN           NaN   
2              2                    1.0                NaN       10000.0   
3              2                    1.0            10000.0       20000.0   
4              3                    1.0                NaN       10000.0   

  Last Pay Raise Date  Variable Pay  Tenure in Org  Performance Rating  \
0                 NaN             0              0                 NaN   
1                 NaN             0              0                 NaN   
2                 NaN             0              0                 9.0   
3          2022-01-01             0              0                 9.5   
4                 NaN             0              0                 9.0   

  Review Date  Engagement Score Engagement Date Effective Date    End Date  
0         NaN        